In [1]:
import redis
import numpy as np
import pandas as pd
import cv2
import FaceRec

C:\Users\nitin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\nitin\anaconda3\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [2]:
hostname='redis-11446.c265.us-east-1-2.ec2.redns.redis-cloud.com'
port=11446
password='73yAJgBY7pYl8gNn9XKpnOSA7ILX2MYP'

r=redis.StrictRedis(host=hostname,port=port,password=password)

In [3]:
r.ping()

True

In [4]:
file_np=np.load('df_students_teacher.npz',allow_pickle=True)
file_np.files
df_1=pd.DataFrame(file_np['arr_0'],columns=file_np['arr_1'])
df_1

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.19881932, 1.4748837, -0.0979588, -0.8805054..."
1,Barack Obama,Teacher,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128..."
2,Chris Evans,Student,"[0.35691047, -0.41491055, 0.039218444, -0.1543..."
3,Morgan Freeman,Teacher,"[0.76278937, 0.890615, 0.46873552, 0.25898102,..."
4,Scarlett Johansson,Student,"[-0.13535985, -0.2821385, -0.42700845, -0.2825..."


In [5]:
df_1['Name_Role']=df_1['Name']+'@'+df_1['Role']
df_1

,Name,Role,Facial_Features,Name_Role
0,Angelina Jolie,Student,"[0.19881932, 1.4748837, -0.0979588, -0.8805054...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.35691047, -0.41491055, 0.039218444, -0.1543...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.76278937, 0.890615, 0.46873552, 0.25898102,...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[-0.13535985, -0.2821385, -0.42700845, -0.2825...",Scarlett Johansson@Student


In [6]:
records=df_1[['Name_Role','Facial_Features']].to_dict(orient='records')
len(records)

5

### Saving Data in Redis

In [7]:
for record in records:
    name_role=record['Name_Role']
    vector=record['Facial_Features']
    
    vector_bytes=vector.tobytes()
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [8]:
import insightface
import cv2
import warnings
warnings.simplefilter('ignore')
from insightface.app import FaceAnalysis
face_app=FaceAnalysis(name='buffalo_l',root='insightface_model')
face_app.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### Register yourself

In [9]:
person_name=input('Enter your name: ')

trials=3
for i in range(trials):
    role=input("""
    Please choose one option:
    1. Employee
    2. Manager
    """)
    
    if role in ['1','2']:
        if role=='1':
            role='Employee'
        else:
            role='Manager'
        key=person_name+'@'+role
        print('Your name:',person_name)
        print('Your role:',role)
        print('Key:',key)
        break
    else:
        print('Invalid entry try again')
        if i==2:
            print('Exceeded maximum trials')




Enter your name: Nirmit

    Please choose one option:
    1. Employee
    2. Manager
    1
Your name: Nirmit
Your role: Employee
Key: Nirmit@Employee


### Collecting Samples

In [10]:
cap=cv2.VideoCapture(0)
face_embeddings=[]
samples=0
while True:
    ret, frame=cap.read()
    if ret==False:
        print('Unable to read camera')
        break
    results=face_app.get(frame,max_num=1)
    for res in results:
        samples+=1
        x1,y1,x2,y2=res['bbox'].astype('int')
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
        embeddings=res['embedding']
        face_embeddings.append(embeddings)
    
    if samples>=200:
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [11]:
len(face_embeddings)

108

In [12]:
x_mean=np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [13]:
x_mean_bytes=x_mean.tobytes()

In [14]:
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1

### Retrieve data from Redis

In [ ]:
retrive_dict=r.hgetall(name='academy:register')
retrive_series=pd.Series(retrive_dict)
retrive_series

In [16]:
retrive_series=retrive_series.apply(lambda x:np.frombuffer(x,dtype=np.float32))
index=retrive_series.index
index=list(map(lambda x:x.decode(),index))

In [17]:
retrive_series.index=index

In [ ]:
retrive_df=retrive_series.to_frame().reset_index()
retrive_df.columns=['name_role','facial_features']
retrive_df[['Name','Role']]=retrive_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrive_df

### Real time prediction

In [19]:
cap=cv2.VideoCapture(0)
p=set()
while True:
    ret, frame=cap.read()
    if ret == False:
        break
    pred_frame=FaceRec.face_prediction(frame, retrive_df,'facial_features',threshold=0.5)
    
    cv2.imshow('frame',frame)
    cv2.imshow('prediction',pred_frame)
    
    results = FaceRec.faceapp.get(frame)
    for r in results:
        x1, y1, x2, y2 = r['bbox'].astype('int')
        embeddings = r['embedding']
        person_name, person_role = FaceRec.search(retrive_df, 'facial_features', test_vector=embeddings, threshold=0.5)
        
        if person_name != 'Unknown' and person_name not in p:
            # Call the speak_name function to speak the detected person's name
            FaceRec.speak_name(person_name)
            p.add(person_name)
        if cv2.waitKey(1)==27:
            break
    if cv2.waitKey(1)==27:
        break
cv2.destroyAllWindows()
cap.release()

In [22]:
cap.release()

In [171]:
#r.flushdb()

True